# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import ipywidgets as widgets

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read the cities.csv to create dataframe, previous cells are not needed to be executed as long as cities.csv was created

vacation_data = pd.read_csv("Resources/Cities by Coordinates.csv", encoding="utf-8")
vacation_data.head()

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Tasiilaq,GL,65.61,-37.64,24.80,68,100,14.99
1,Ushuaia,AR,-54.80,-68.30,39.20,64,40,9.17
2,Vardø,NO,70.37,31.11,28.11,93,74,13.85
3,Pevek,RU,69.70,170.31,9.97,91,4,14.23
4,Coquimbo,CL,-29.95,-71.34,57.20,87,74,5.82


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

In [4]:

# Convert Humidity to float and store
# remove NaN values
vacation_data = vacation_data.dropna()
humidity = vacation_data["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = vacation_data[["Latitude", "Longitude"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                dissipating=False, max_intensity=max(vacation_data["Humidity"]),                                
                point_radius = 2)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
city_df = vacation_data.loc[(vacation_data["Temperature"] > 70) & (vacation_data["Temperature"] < 80) & 
                            (vacation_data["Cloudiness"] == 0), :]
city_df = city_df.dropna(how='any')
city_df.reset_index(inplace=True)
del city_df['index']
city_df.head()

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Cuajinicuilapa,MX,16.47,-98.42,72.84,83,0,2.77
1,Richards Bay,ZA,-28.78,32.04,72.88,75,0,12.82
2,Bagotville,AU,-28.98,153.42,70.75,56,0,12.75
3,Brufut,GM,13.38,-16.75,71.44,80,0,7.76
4,Shiyan,CN,32.65,110.78,72.07,19,0,6.93


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = city_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: Cuajinicuilapa.
Closest hotel is Pozo S. XVI.
------------
Retrieving Results for Index 1: Richards Bay.
Closest hotel is BON Hotel Waterfront Richards Bay.
------------
Retrieving Results for Index 2: Bagotville.
Closest hotel is Broadwater Sunrise Caravan Park.
------------
Retrieving Results for Index 3: Brufut.
Closest hotel is Sheraton Gambia Hotel Resort & Spa.
------------
Retrieving Results for Index 4: Shiyan.
Closest hotel is Karen International Hotel.
------------
Retrieving Results for Index 5: Nangong.
Closest hotel is 7Days Inn Xingtai South Yucai Road.
------------
Retrieving Results for Index 6: Pingdingshan.
Closest hotel is Chengfu Hotel.
------------
Retrieving Results for Index 7: Taoudenni.
Missing field/result... skipping.
------------
Retrieving Results for Index 8: Inhambane.
Closest hotel is MHL Turismo SA.
------------
Retrieving Results for Index 9: Pingshan.
Closest hotel is Island Skylight Resort.
------------
Retrieving Resu

In [10]:
hotel_df

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,Hotel Name
0,Cuajinicuilapa,MX,16.47,-98.42,72.84,83,0,2.77,Pozo S. XVI
1,Richards Bay,ZA,-28.78,32.04,72.88,75,0,12.82,BON Hotel Waterfront Richards Bay
2,Bagotville,AU,-28.98,153.42,70.75,56,0,12.75,Broadwater Sunrise Caravan Park
3,Brufut,GM,13.38,-16.75,71.44,80,0,7.76,Sheraton Gambia Hotel Resort & Spa
4,Shiyan,CN,32.65,110.78,72.07,19,0,6.93,Karen International Hotel
5,Nangong,CN,37.36,115.37,74.39,17,0,16.67,7Days Inn Xingtai South Yucai Road
6,Pingdingshan,CN,33.74,113.30,73.09,19,0,15.52,Chengfu Hotel
7,Taoudenni,ML,22.68,-3.98,75.63,18,0,3.44,NaN
8,Inhambane,MZ,-23.86,35.38,75.20,78,0,5.68,MHL Turismo SA
9,Pingshan,CN,22.99,114.71,73.99,45,0,3.00,Island Skylight Resort


In [12]:
# NOTE: Do not change any of the code in this cell

locations = hotel_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))